In [15]:
# 3.3 线性回归的简洁实现

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

print("features.shape:", features.shape)
print("labels.shape:", labels.shape)

features.shape: torch.Size([1000, 2])
labels.shape: torch.Size([1000, 1])


In [2]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [3]:
next(iter(data_iter))

[tensor([[ 0.0784,  1.0700],
         [-0.9819, -1.0927],
         [-1.1853,  0.8790],
         [-0.0033,  0.2201],
         [-1.0009, -1.1610],
         [ 0.6782,  0.8495],
         [-0.3446,  0.2833],
         [ 0.5708, -0.1599],
         [ 0.3851,  0.4674],
         [-1.1538, -1.0720]]),
 tensor([[ 0.7200],
         [ 5.9568],
         [-1.1584],
         [ 3.4494],
         [ 6.1525],
         [ 2.6518],
         [ 2.5571],
         [ 5.8746],
         [ 3.3818],
         [ 5.5373]])]

In [11]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

In [20]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [21]:
loss = nn.MSELoss(reduction='sum')
#loss = nn.MSELoss()

In [24]:
#trainer = torch.optim.SGD(net.parameters(), lr=0.03)
trainer = torch.optim.SGD(net.parameters(), lr=0.003)

In [25]:
# data_iter中的loss是指一个批次的损失。
# trainer.zero_grad()是梯度清零
# l.backward()是求解这一个批次的样本的导数和
# trainer.step()# 以求得的导数和，结合优化器，更新参数 w和b，然后进行下一批次的训练

num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.111062
epoch 2, loss 0.105317
epoch 3, loss 0.105325


In [16]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([ 1.7270, -2.7868])
b的估计误差： tensor([3.5079])


In [27]:
print(net[0].weight.grad)
print(net[0].bias.grad)

tensor([[-0.0773, -0.0493]])
tensor([-0.0291])
